<a href="https://colab.research.google.com/github/FabianaAndrade/projeto_IA/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   Aryane de Alcantara Chaves - 11893303
*   Fabiana Andrade Barroso - 13729431
*   Gabriel Kennuy de Assis Malta Peruso - 13673173
*   Ingrid Moreno da Silva - 13729070
*   Izabel Christine dos Santos Barranco  - 11847711
*   Jenifer Galvão de Morais - 11912147









In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import confusion_matrix

In [ ]:
# 1. Ler os dados dos arquivos
X_file = 'X.txt'
Y_file = 'Y_letra.txt' # pular linha no final do arquivo quando for subir

with open(X_file, 'r') as file:
    linhas_X = file.readlines()

with open(Y_file, 'r') as file:
    rotulos = file.readlines()

# 2. Codificação dos rótulos (Y_letra.txt)
classes = sorted(set(rotulos))  # Obter todas as classes únicas
num_classes = len(classes) # Usar este valor para o parametro de quantidade de neuronios de saida

rotulos_encoded = []
for rotulo in rotulos:
    index = classes.index(rotulo)  # Obter o índice da classe no vetor de classes
    one_hot = [0] * num_classes
    one_hot[index] = 1  # Definir o valor 1 para a classe correspondente
    rotulos_encoded.append(one_hot)

x = []
for linha in linhas_X:
    valores = linha.strip().split(',')
    valores_int = [int(valor) for valor in valores if valor.strip()]  # Remover valores vazios e espaços em branco
    if valores_int:
        x.append(valores_int)


x = np.array(x, dtype=int)  # Convertendo para int
y = np.array(rotulos_encoded)
tamanho_vetor = len(x[0]) # Usar esse valor para o parametro de quantidade de neuronios de entrada

print("x ", x[0])
print("y ", y[26])

In [ ]:
class MLP:
    # quantidade de neuronios de entrada = quantidade de valores do vetor/matriz do input
    # quantidade de neuronios da camada escondida = pode ser um parametro
    # quantidade de neuronios da camada de saida = quantidade de labels existentes do problema extraidas do arquivo y_letra
    def __init__(self, tamanho_camada_entrada, tamanho_camada_escondida, tamanho_camada_saida, taxa_aprendizado):

        # Inicialização aleatória dos pesos e bias a partir de uma distribuição normal (gaussiana) com média 0 e desvio padrão 1.
        self.pesos_saindo_camada_de_entrada_para_escondida = np.random.rand(tamanho_camada_entrada, tamanho_camada_escondida)
        self.pesos_saindo_camada_escondida_para_saida = np.random.rand(tamanho_camada_escondida, tamanho_camada_saida)

        # Inicialização dos bias
        self.bias_camada_escondida = np.random.rand(tamanho_camada_escondida)
        self.bias_camada_saida = np.random.rand(tamanho_camada_saida)

        self.taxa_aprendizado = taxa_aprendizado

    def sigmoide(self, x):
        return 1 / (1 + np.exp(-x))

    def derivada_sigmoide(self, x):
        return x * (1 - x)

    def forward(self, X):
        # Calculo do input da camada escondida
        # Calcula atraves da multiplicação do valores da camada de entrada com o pesos da camada de entrada para a camada escondida
        # e soma com os bias da camada de entrada para a camada escondida
        self.input_camada_escondida = np.dot(X, self.pesos_saindo_camada_de_entrada_para_escondida) + self.bias_camada_escondida
        self.output_camada_escondida = self.sigmoide(self.input_camada_escondida)

        # Calculo da camada escondida
        # Calcula atraves do resultado do input da camada escondida passando pela função sigmoide
        self.input_camada_saida = np.dot(self.output_camada_escondida, self.pesos_saindo_camada_escondida_para_saida) + self.bias_camada_saida
        self.saida = self.sigmoide(self.input_camada_saida)

        return self.saida

    def backward(self, X, y, saida):
        # Backward pass da saída para a camada escondida
        erro = y - saida
        delta_saida = erro * self.derivada_sigmoide(saida)

        # Backward pass da camada escondida para a entrada
        camada_escondida_erro = np.dot(delta_saida, self.pesos_saindo_camada_escondida_para_saida.T)
        camada_escondida_delta = camada_escondida_erro * self.derivada_sigmoide(self.output_camada_escondida)

        # Atualização dos pesos e bias usando gradiente descendente
        self.pesos_saindo_camada_escondida_para_saida += self.taxa_aprendizado * np.dot(self.output_camada_escondida.T, delta_saida)
        self.pesos_saindo_camada_de_entrada_para_escondida += self.taxa_aprendizado * np.dot(X.T, camada_escondida_delta)
        self.bias_camada_saida += self.taxa_aprendizado * np.sum(delta_saida, axis=0)
        self.bias_camada_escondida += self.taxa_aprendizado * np.sum(camada_escondida_delta, axis=0)

    def treinamento(self, X, y, epocas):
        erros = []
        for epoca in range(epocas):
            saida = self.forward(X)
            self.backward(X, y, saida)
            eqm = np.mean((y - saida) ** 2)
            erros.append(eqm)
            print(f'Epoca {epoca+1}/{epocas}, Erro quadratico medio: {eqm}')
        return erros

    def acuracia(self, X, y):
        saida = self.forward(X)
        preditos = np.round(saida)
        acuracia = np.mean(preditos == y)
        return acuracia

In [ ]:
# Define o diretório de output

output_dir = "output_files"
# Cria diretório de output
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Função criada para salvar os pesos iniciais no .txt

def save_initial_weights(initial_weights):
    # Define o caminho e o nome do arquivo de saída
    output_file = "initial_weights.txt"

    # Abre o arquivo para escrita
    with open(output_file, "w") as txt_file:
        # Escreve um cabeçalho indicando a origem dos pesos
        txt_file.write("Pesos Iniciais:\n\n")

        # Itera sobre cada camada de pesos
        for index, layer in enumerate(initial_weights):
            # Determina a origem da camada de pesos
            if index == 0:
                txt_file.write("Passagem da camada sensorial para a camada escondida\n\n")
            elif index == 1:
                txt_file.write("Passagem da camada escondida para a camada de saída\n\n")

            # Itera sobre os neurônios na camada
            for neuron_index, weights in enumerate(layer):
                # Escreve os pesos do neurônio
                txt_file.write("Neurônio: " + str(neuron_index + 1) + ":\n")
                for weight in weights:
                    txt_file.write(str(weight) + " ")
                txt_file.write("\n")

            # Adiciona uma linha em branco entre as camadas
            txt_file.write("\n")

In [ ]:
# Função criada para salvar os pesos finais no .txt

def save_final_weights(final_weights):
    # Define o caminho e o nome do arquivo de saída
    output_file = "final_weights.txt"

    # Abre o arquivo para escrita
    with open(output_file, "w") as txt_file:
        # Escreve um cabeçalho indicando a origem dos pesos
        txt_file.write("Pesos Finais:\n\n")

        # Itera sobre cada camada de pesos
        for index, layer in enumerate(final_weights):
            # Determina a origem da camada de pesos
            if index == 0:
                txt_file.write("Passagem da camada sensorial para a camada escondida\n\n")
            elif index == 1:
                txt_file.write("Passagem da camada escondida para a camada de saída\n\n")

            # Itera sobre os neurônios na camada
            for neuron_index, weights in enumerate(layer):
                # Escreve os pesos do neurônio
                txt_file.write("Neurônio: " + str(neuron_index + 1) + ":\n")
                for weight in weights:
                    txt_file.write(str(weight) + " ")
                txt_file.write("\n")

            # Adiciona uma linha em branco entre as camadas
            txt_file.write("\n")

In [ ]:
# Função criada para salvar os parametros no .txt

def save_architecture_parameters(learning_rate, stopping_criteria, input_neurons, hidden_neurons, output_neurons):
    # Define o caminho e o nome do arquivo de saída
    output_file = "architecture_parameters.txt"

    # Abre o arquivo para escrita
    with open(output_file, "w") as txt_ParametersFile:
        txt_ParametersFile.write("Taxa de aprendizado: " +
                        str(learning_rate) + "\n")
        txt_ParametersFile.write("Critério de parada: " +
                        str(stopping_criteria) + "\n")
        txt_ParametersFile.write("Nro de neurônios (camada de entrada): " +
                        str(input_neurons) + "\n")
        txt_ParametersFile.write("Nro de neurônios (camada oculta): " +
                        str(hidden_neurons) + "\n")
        txt_ParametersFile.write("Nro de neurônios (camada de saída): " +
                        str(output_neurons) + "\n")
    txt_ParametersFile.close()
    # Fecha o arquivo

In [ ]:
# Função criada para salvar os erros por epoca no .txt

def save_error_by_epoch(error, epoch):
    # Define o caminho e o nome do arquivo de saída
    output_file = "error_by_epoch.txt"

    # Abre o arquivo para escrita
    with open(output_file, "a") as txt_file:
        # Escreve a média de erro da época
        txt_file.write("Média de erro da época " + str(epoch) + ": " +
                        str(error) + "\n")

    # Não é preciso fechar pois estamos usando appende ao inves de write